In [ ]:
import rospy

In [ ]:
DEFAULT_I = 10
try:
    i = rospy.get_param('ii')
except:
    print('can not collect param. Going to use default value.')
    i = DEFAULT_I
    
print(i)

In [1]:
from pcg_gazebo.task_manager import ProcessManager
from pcg_gazebo.generators.creators import create_models_from_config

TARGET_SIM = dict(
    ros_host='localhost',
    ros_port=11311,
    gazebo_host='localhost',
    gazebo_port=11345
)

process_manager = ProcessManager(**TARGET_SIM)
gazebo_proxy = process_manager.get_gazebo_proxy()
print(gazebo_proxy.ros_config)

ROS_MASTER_URI=http://localhost:11311, GAZEBO_MASTER_URI=http://localhost:11345


In [79]:
## spawn box to the any place of wall collision space coordinates

import numpy as np
from pcg_gazebo.generators.shapes import random_rectangle, \
    random_points_to_triangulation, random_rectangles
import trimesh

THRESHOLD_WALL = 0.7
ROOM_SIZE_RADIUS = 4.5
ROOM_WALL_HEIGHT = 0.8
OBSTACLE_NUM = 10
OBSTACLE_SIZE = 0.8
TARGET_SIZE = 0.4

config = [
    dict(
        type='extrude',
        args=dict(
            polygon=random_points_to_triangulation(
                x_min=-ROOM_SIZE_RADIUS,
                x_max=ROOM_SIZE_RADIUS,
                y_min=-ROOM_SIZE_RADIUS,
                y_max=ROOM_SIZE_RADIUS
            ),
            height=ROOM_WALL_HEIGHT,
            thickness=0.1,
            extrude_boundaries=True,
            name='extruded_poly_triangulation',
            color='random'
        )
    )
]

models = create_models_from_config(config)
print(models)
for model in models:
    print(model.get_link_by_name('visual_mesh'))
    print(model.links is None)
    for elem in model.links:
        cols = model.get_link_by_name(elem).collisions
        print(cols[0])
        col_mesh = cols[0]._geometry._geometry_entity.get_meshes()[0]
        sample_pts = cols[0]._geometry._geometry_entity.get_samples(OBSTACLE_NUM)
        print(sample_pts)
    model.spawn(
        gazebo_proxy=gazebo_proxy, 
        robot_namespace=model.name,
        pos=[0,0,ROOM_WALL_HEIGHT/2]
    )

print(len(col_mesh.vertices))
mini_col_mesh = trimesh.Trimesh(np.array(col_mesh.vertices)*THRESHOLD_WALL)

sample_pts = np.array(mini_col_mesh.convex_hull.sample(len(sample_pts)))
sample_pts += cols[0].pose.position * THRESHOLD_WALL
sample_pts[:,2] = OBSTACLE_SIZE/2

sample_rot = np.zeros([len(sample_pts), 3])
sample_rot[:,2] = np.random.random([len(sample_pts)])*np.pi*2

sizes = [[OBSTACLE_SIZE, OBSTACLE_SIZE, OBSTACLE_SIZE]]*len(sample_pts)

config_obstacle_box = [
    dict(
        type='box_factory',
        args=dict(
            size=sizes,
            name='obstacle_box',
            use_permutation=True,
            color='white'
        )
    )
]

models_box = create_models_from_config(config_obstacle_box)
for model, p, r in zip(models_box, sample_pts, sample_rot):
    print(p, r)
    model.spawn(
        gazebo_proxy=gazebo_proxy, 
        robot_namespace=model.name,
        pos=list(p),
        rot=list(r)
    )

None
False
<collision name="collision">
  <geometry>
    <mesh>
      <scale>1 1 1</scale>
      <uri>file:///root/.pcg/meshes/YEcTxqdIlR.stl</uri>
    </mesh>
  </geometry>
  <max_contacts>20</max_contacts>
  <pose frame="">-1.1267 -1.06112 -0.4 0 -0 0</pose>
</collision>

[[ 0.8446597   4.31495609  0.4923411 ]
 [-1.32939889  3.62870235  0.56645934]
 [-1.30136098  3.42762894  0.6554246 ]
 [ 4.15158405  0.6249689   0.60124047]
 [-2.25607361  2.96292256  0.25897758]
 [ 3.79076431  3.70934131  0.79390184]
 [-1.34340688  3.44389602  0.8       ]
 [ 4.2097089   3.31342802  0.34247602]
 [ 1.94731166  4.29594652  0.47039146]
 [-1.04306018 -1.26439227  0.05292209]]
196
(array([2.10646239, 0.73893361, 0.4       ]), array([0.       , 0.       , 1.8701774]))
(array([-0.86905077,  0.58625451,  0.4       ]), array([0.        , 0.        , 0.56786298]))
(array([ 0.63791376, -0.15877169,  0.4       ]), array([0.        , 0.        , 1.88557017]))
(array([ 1.3171567 , -1.14280914,  0.4       ]), array

In [80]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=np.sqrt(OBSTACLE_SIZE*2), min_samples=1).fit(np.array(sample_pts))
labels = db.labels_
print(labels)


[0 1 0 0 1 1 2 1 1 0]


In [81]:
sizes = [[TARGET_SIZE, TARGET_SIZE, TARGET_SIZE]]*(max(labels)+1)
config_target_box = [
    dict(
        type='box_factory',
        args=dict(
            size=sizes,
            name='target_box',
            use_permutation=True,
            color='cyan'
        )
    )
]

models_target = create_models_from_config(config_target_box)

In [82]:
def get_square_horizon(base_pos, half_length):
    poss = [None]*4
    for i in range(4):
        pos = [0,0,base_pos[2]]
        d1 = [1,1,-1,-1]
        d2 = [1,-1,-1,1]
        pos[0] = base_pos[0] + d1[i]*half_length
        pos[1] = base_pos[1] + d2[i]*half_length
        poss[i] = pos
    
    return poss

def get_extended_face(face_vertices, length):
    face_bottom = np.copy(face_vertices)
    face_bottom[:,2] += length
    return np.concatenate([face_vertices, face_bottom])

#### Generate convex hull with cluster

- Origin version
- Expanded version

To create expand version is the steps below.

1. preparate more than 3 points per cluster
2. calculate that polygon's gravity and vectors from this gravity each point
3. extend vector length with depend on box size
4. create convex hull as expanded version

In [89]:
hull_points_2d = [None]*(max(labels)+1)
for i in range(len(hull_points_2d)):
    pts = sample_pts[labels==i]
    if len(pts) < 2:
        v = []
        for p in pts:
            v.extend(get_square_horizon(p, 0.01))
        pts = np.array(v)
        
#     convex_2d = ConvexHull(pts[:,:2])
    hull_points_2d[i] = trimesh.convex.hull_points(pts[:,:2])


zone_hull = [None]*(len(hull_points_2d)) # trimesh.Trimesh in
for i in range(len(zone_hull)):
    tmp = np.zeros([len(hull_points_2d[i]), 3])
    tmp[:,:2] = hull_points_2d[i]
    pts = get_extended_face(tmp, OBSTACLE_SIZE)
    zone_hull[i] = trimesh.Trimesh(vertices=pts).convex_hull
    
    
zone_hull_expand = [None]*(len(hull_points_2d)) # trimesh.Trimesh in
zone_vecs_g2v = [None]*(len(hull_points_2d)) # normalized vector gravity to vertice
hull_points_2d_expand = [None]*(len(hull_points_2d))
for i in range(len(zone_hull_expand)):
    g = np.mean(hull_points_2d[i], axis=0)
#     print('g,', g)
#     print('norm', np.linalg.norm(hull_points_2d[i]-g, axis=1))
#     print('2dp', hull_points_2d[i]-g)
    norm = np.linalg.norm(hull_points_2d[i]-g, axis=1)
    zone_vecs_g2v[i] = (hull_points_2d[i]-g) / np.array([norm]).T ## KETAOTI OKIRUYO
    hull_points_2d_expand[i] = hull_points_2d[i]+zone_vecs_g2v[i]*(np.sqrt(OBSTACLE_SIZE*2)/2)
    
    tmp = np.zeros([len(hull_points_2d[i]), 3])
    tmp[:,:2] = hull_points_2d_expand[i]
    pts = get_extended_face(tmp, OBSTACLE_SIZE)
    zone_hull_expand[i] = trimesh.Trimesh(vertices=pts).convex_hull
    
print("zone_vecs_g2v", zone_vecs_g2v)
print("zone_full: \n{}".format(zone_hull[0].mass))
print("zone_full_expand: \n{}".format(zone_hull_expand[0].mass))

print(hull_points_2d_expand)


('zone_vecs_g2v', [array([[-0.97744593, -0.21118584],
       [ 0.01734616, -0.99984954],
       [ 0.73094496,  0.68243642],
       [-0.31187443,  0.95012333]]), array([[-0.68217415,  0.73118974],
       [-0.9302495 , -0.36692762],
       [-0.01773493, -0.99984272],
       [ 0.95260975, -0.30419511],
       [ 0.92270819,  0.38549915]]), array([[-0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678],
       [-0.70710678,  0.70710678]])])
zone_full: 
1.03125814624
zone_full_expand: 
3.16212451949
[array([[ 0.01972268, -0.29233734],
       [ 1.32812738, -1.77516952],
       [ 2.56875257,  1.1705443 ],
       [ 0.93166875,  1.09860582]]), array([[-2.37557052,  1.86224466],
       [-2.69467194,  0.18748527],
       [-1.34148683, -0.52435283],
       [-0.26656746,  0.39386463],
       [ 0.23716283,  1.38026164]]), array([[-1.97255174, -1.71531067],
       [-1.05812455, -1.71531067],
       [-1.05812455, -0.80088348],
       [-1.97255174, -0.80088348]])

#### Methods to get target position: 

- sample a vector in convex face that generated with all origin position of cubes in a cluster

In [17]:
zone_samples = [None]*len(zone_hull)
for i in range(len(zone_samples)):
    ## sample_surface return tuple, so choice 0 index
    samples = trimesh.sample.sample_surface(zone_hull[i], 100)[0]
    zone_samples[i] = samples[
        np.logical_and(
            np.not_equal(samples[:,2], 0), 
            np.not_equal(samples[:,2], OBSTACLE_SIZE)
        )
    ]

choiced_goal = np.zeros([len(zone_hull),3])
for i in range(len(choiced_goal)):
    choiced_goal[i] = zone_samples[i][np.random.choice(range(len(zone_samples[i])))]
    
choiced_goal[:,2] = TARGET_SIZE/2

print(choiced_goal)
for model, p in zip(models_target, choiced_goal):
    model.spawn(
        gazebo_proxy=gazebo_proxy, 
        robot_namespace=model.name,
        pos=list(p*1.4)
    )

[[ 1.6280383   0.47532221  0.4       ]
 [ 1.06612518 -1.25524001  0.4       ]
 [ 1.07051614 -0.37006875  0.4       ]
 [ 1.6280383   0.47532221  0.        ]
 [ 1.06612518 -1.25524001  0.        ]
 [ 1.07051614 -0.37006875  0.        ]]
[[-1.91878831 -1.45287269  0.4       ]
 [-1.13075492 -1.3815387   0.4       ]
 [-1.72196411 -1.775725    0.4       ]
 [-1.78581065 -1.93064327  0.4       ]
 [-0.8571042  -1.08652255  0.4       ]
 [-1.91878831 -1.45287269  0.        ]
 [-1.13075492 -1.3815387   0.        ]
 [-1.72196411 -1.775725    0.        ]
 [-1.78581065 -1.93064327  0.        ]
 [-0.8571042  -1.08652255  0.        ]]
[[1.52151552 2.82872883 0.4       ]
 [1.52151552 2.82872883 0.        ]]
[[-0.62495616  0.95918874  0.4       ]
 [-0.62495616  0.95918874  0.        ]]
zone_full: 
[<trimesh.Trimesh(vertices.shape=(6, 3), faces.shape=(8, 3))>, <trimesh.Trimesh(vertices.shape=(8, 3), faces.shape=(12, 3))>, <trimesh.Trimesh(vertices.shape=(8, 3), faces.shape=(12, 3))>, <trimesh.Trimesh(vert

- sample a vector in above convex face that expanded with depend on target size

In [90]:
zone_samples = [None]*len(zone_hull_expand)
for i in range(len(zone_samples)):
    ## sample_surface return tuple, so choice 0 index
    samples = trimesh.sample.sample_surface(zone_hull_expand[i], 100)[0]
    zone_samples[i] = samples[
        np.logical_and(
            np.not_equal(samples[:,2], 0), 
            np.not_equal(samples[:,2], OBSTACLE_SIZE)
        )
    ]

choiced_goal = np.zeros([len(zone_hull_expand),3])
for i in range(len(choiced_goal)):
    choiced_goal[i] = zone_samples[i][np.random.choice(range(len(zone_samples[i])))]
    
choiced_goal[:,2] = TARGET_SIZE/2

print(choiced_goal)
for model, p in zip(models_target, choiced_goal):
    model.spawn(
        gazebo_proxy=gazebo_proxy, 
        robot_namespace=model.name,
        pos=list(p)
    )

[[ 1.01877164  1.10243339  0.2       ]
 [-2.2190668  -0.06270508  0.2       ]
 [-1.05812455 -1.41798253  0.2       ]]


In [100]:
def delete_models(model_name, start=None, stop=None):
    if start is not None and stop is not None:
        for i in range(start, stop+1):
            gazebo_proxy.delete_model(model_name=model_name+"_"+str(i))
    else:
        gazebo_proxy.delete_model(model_name='')
        
delete_models('box_static_var_size', 0, 10)